In [1]:
import random
import numpy as np
from collections import deque, namedtuple
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.datasets.samples_generator import make_blobs
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

c:\python\python37\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [9]:
class Vertice:
    
    def __init__(self, key=None, colour=None, height=None , x=None, y=None):
        self.key = key
        self.colour = colour
        self.height = height
        self.neighbours = {}
        self.x = x
        self.y = y 
        
    def add_neighbour(self, node, weight):
        self.neighbours[node] = weight
    
    def add_xy(self, x, y):
        self.x = x
        self.y = y
    
    def add_colour(self, colour):
        self.colour = colour
    
    def add_height(self, height):
        self.height = height
    
    def get_neighbours(self):
        return self.neighbours.keys()
    
    def get_key(self):
        return self.key
    
    def get_weight(self, node):
        return self.neighbours[node]       
    

class Network:
    
    def __init__(self, n):
        self.height = n + 2
        self.vertices = {}
        self.amount = 0
        self.make_ladder(n)
        self.start = self.get_vertice(1)
        self.ends = [self.get_vertice(self.amount - 1), self.get_vertice(self.amount)]
        self.regime = self.make_regime()
        self.make_heights(self.regime)
            
    def make_ladder(self, n):        
        self.add_edge(1, 'red', 2, 'blue', random.randint(1,9))
        self.add_edge(1, 'red', 3, 'red', random.randint(1,9))
        row = 0
        for i in range(2, n*2+2, 2):
            if row%2 == 0:
                self.add_edge(i, 'blue', i+2, 'red', random.randint(1,9))
                self.add_edge(i+1, 'red', i+2, 'red', random.randint(1,9))
                self.add_edge(i, 'blue', i+3, 'blue', random.randint(1,9))
                self.add_edge(i+1, 'red', i+3, 'blue', random.randint(1,9))
            else: 
                self.add_edge(i, 'red', i+2, 'blue', random.randint(1,9))
                self.add_edge(i+1, 'blue', i+2, 'blue', random.randint(1,9))
                self.add_edge(i, 'red', i+3, 'red', random.randint(1,9))
                self.add_edge(i+1, 'blue', i+3, 'red', random.randint(1,9))
            row += 1
        self.add_edge(n*2, 'red', n*2+2, self.get_vertice(n*2).colour, random.randint(1,9))
        self.add_edge(n*2+1, 'blue', n*2+2, self.get_vertice(n*2).colour, random.randint(1,9))
        
    def make_regime(self):
        n_components = 120
        X, truth = make_blobs(n_samples=300, centers=n_components, 
                              cluster_std = [0.002, 0.001, 0.005, 0.001, 0.0011,0.002,0.004,0.0013,0.001,0.008,0.005,0.003,0.006,0.004,0.001, 0.002, 0.001, 0.005, 0.001, 0.0011,0.002,0.004,0.0013,0.001,0.008,0.005,0.003,0.006,0.004,0.001, 0.002, 0.001, 0.005, 0.001, 0.0011,0.002,0.004,0.0013,0.001,0.008,0.005,0.003,0.006,0.004,0.001, 0.002, 0.001, 0.005, 0.001, 0.0011,0.002,0.004,0.0013,0.001,0.008,0.005,0.003,0.006,0.004,0.001,0.002, 0.001, 0.005, 0.001, 0.0011,0.002,0.004,0.0013,0.001,0.008,0.005,0.003,0.006,0.004,0.001, 0.002, 0.001, 0.005, 0.001, 0.0011,0.002,0.004,0.0013,0.001,0.008,0.005,0.003,0.006,0.004,0.001, 0.002, 0.001, 0.005, 0.001, 0.0011,0.002,0.004,0.0013,0.001,0.008,0.005,0.003,0.006,0.004,0.001, 0.002, 0.001, 0.005, 0.001, 0.0011,0.002,0.004,0.0013,0.001,0.008,0.005,0.003,0.006,0.004,0.001], 
                              random_state=57,center_box=(0,1))

        # Extract x and y
        x = X[:, 0]
        y = X[:, 1]

        # Define the borders
#         deltaX = (max(x) - min(x))/10
#         deltaY = (max(y) - min(y))/10
#         xmin = min(x) - deltaX
#         xmax = max(x) + deltaX
#         ymin = min(y) - deltaY
#         ymax = max(y) + deltaY

        # Create meshgrid
        xx, yy = np.mgrid[0:1:100j, 0:1:100j]

        #######
        positions = np.vstack([xx.ravel(), yy.ravel()])
        values = np.vstack([x, y])
        kernel = st.gaussian_kde(values)

        f = np.reshape(kernel(positions).T, xx.shape)
        return f    
        
    def make_heights(self, reg, left =.1, right=.9):    

        self.start.add_height(reg[int(0.1*100)][int(0.5*100)]*100 + random.randint(-10,10))
        
        spacing = (right - left) / (self.height - 1)
        for n in range(2, self.amount, 2):
            self.get_vertice(n).add_height(reg[int((n/2 * spacing + left)*100)][int(0.8*100)]*100 + random.randint(-10,10))
            self.get_vertice(n+1).add_height(reg[int((n/2 * spacing + left)*100)][int(0.2*100)]*100 + random.randint(-10,10))
            
        for i in range(1, self.amount + 1):
            node = self.get_vertice(i)
            for neighbour in node.get_neighbours():
                node.neighbours[neighbour] = abs(neighbour.height - node.height)
             
    def path_greedy(self, node):
        cost = 0
        path = [1]
        colour = [node.colour]
        while node not in self.ends :
            neighbours = [(neighbour, node.get_weight(neighbour)) for neighbour in list(node.get_neighbours())]
            min_node = [neighbours[0]]
            for i in range(1, len(neighbours)):
                if min_node[0][1] > neighbours[i][1]:
                    min_node = [neighbours[i]]
                elif min_node[0][1] == neighbours[i][1]:
                    min_node.append(neighbours[i])
                else:
                    continue
            
            new_node = random.choice(min_node)[0]
            path.append(new_node.get_key())
            colour.append(new_node.colour)
            cost += self.diff_height(node, new_node)
            node = new_node
        
        for col in colour:
            if col == 'blue':
                cost += 5
            else:
                cost -= 1
        
        return cost, path
    
    def generate_greedy(self, node, n, colour, function):
        left_node = []
        right_node = []
        left_colour = []
        right_colour = []
        picked_node = []
        
        cost = 0
        for _ in range(n):
            self.make_heights(self.regime)
            self.random_colours(colour)
            while node not in self.ends:
                
                l_node = list(node.get_neighbours())[0]
                r_node = list(node.get_neighbours())[1]
                
                left_node.append(node.get_weight(l_node))
                right_node.append(node.get_weight(r_node))
                left_colour.append(l_node.colour)
                right_colour.append(r_node.colour)
                
                chosen_node = self.choose_node(node.get_weight(l_node), l_node.colour, node.get_weight(r_node), r_node.colour, function)
                picked_node.append(chosen_node)
                
                if chosen_node:
                    node = r_node
                else:
                    node = l_node
            node = self.start
        
        data = {
            'left_node': left_node,
            'right_node': right_node, 
            'left_colour': left_colour,
            'right_colour': right_colour,
            'picked_node': picked_node
        }
        
        df = pd.DataFrame(data)
        df.to_csv(str(function) + '_' + str(int(self.amount/2)*n) + '_' + str(colour) + '.csv', encoding='utf-8', index=False)
#         print(df)
        
    def creating_all_data(self, n):
    
        for i in range(2, 9):
            self.generate_greedy(self.start, n, i)
    
    def choose_node(self, l_weight, l_colour, r_weight, r_colour, function):
        if function == 'linear':
            if self.cost_linear(l_weight, l_colour) <= self.cost_linear(r_weight, r_colour):
                return 0
            else:
                return 1
        if function == 'non-linear':
            if self.cost_non_linear(l_weight, l_colour) <= self.cost_non_linear(r_weight, r_colour):
                return 0
            else:
                return 1
        if function == 'non-trans':
            pref_dic = {'red': {'red': 0, 'blue': 1, 'green': -1, 'yellow': -1, 'purple': 1, 'white': 1, 'black': 0, 'orange': -1},
                        'blue': {'red': -1, 'blue': 0, 'green': 1, 'yellow': 1, 'purple': 0, 'white': -1, 'black': -1, 'orange': -1}, 
                        'green': {'red': 1, 'blue': -1, 'green': 0, 'yellow': 0, 'purple': -1, 'white': 1, 'black': 1, 'orange': 1}, 
                        'yellow': {'red': 1, 'blue': -1, 'green': 0, 'yellow': 0, 'purple': -1, 'white': -1, 'black': 1, 'orange': 1}, 
                        'purple': {'red': -1, 'blue': 0, 'green': 1, 'yellow': 1, 'purple': 0, 'white': 1, 'black': -1, 'orange': 1}, 
                        'white': {'red': -1, 'blue': 1, 'green': -1, 'yellow': 1, 'purple': -1, 'white': 0, 'black': -1, 'orange': 0}, 
                        'black': {'red': 0, 'blue': 1, 'green': -1, 'yellow': -1, 'purple': 1, 'white': 1, 'black': 0, 'orange': -1}, 
                        'orange': {'red': 1, 'blue': 1, 'green': -1, 'yellow': -1, 'purple': -1, 'white': 0, 'black': 1, 'orange': 0}}
            
            pref = pref_dic[l_colour][r_colour]
            if pref == 0:
                if l_weight <= r_weight:
                    return 0
                else:
                    return 1
            elif pref == 1:
                return 0
            else:
                return 1     
                
    def cost_linear(self, height, colour):
        if colour == 'red':
            return height + 5
        elif colour == 'blue':
            return height - 5
        elif colour == 'green':
            return height + 10
        elif colour == 'yellow':
            return height - 10
        elif colour == 'purple':
            return height + 15
        elif colour == 'white':
            return height - 15
        elif colour == 'black':
            return height + 25
        else:
            return height - 25
    
    def cost_non_linear(self, height, colour):
        dic = {'red': 1.2, 'blue': 0.8, 'green': 2, 'yellow': 0.5, 'purple': 1.3, 'white': 1.1, 'black': 0.6, 'orange': 1.5}
        if colour == 'red':
            return height**dic['red']
        elif colour == 'blue':
            return height**dic['blue']
        elif colour == 'green':
            return height**dic['green']
        elif colour == 'yellow':
            return height**dic['yellow']
        elif colour == 'purple':
            return height**dic['purple']
        elif colour == 'white':
            return height**dic['white']
        elif colour == 'black':
            return height**dic['black']
        else:
            return height**dic['orange']
        
    def random_colours(self, amount):
        colours = ['blue', 'red', 'green', 'yellow', 'purple', 'white', 'black', 'orange'][:amount]
        for i in range(1, self.amount + 1):
            self.get_vertice(i).add_colour(random.choice(colours)) 
    
    def diff_height(self, start, end): 
        return abs(end.height - start.height)
        
    def add_vertice(self, key,colour):
        node = Vertice(key, colour)
        self.vertices[key] = node
        self.amount += 1
    
    def add_edge(self, key1, col1, key2, col2, weight):
        if key1 not in self.vertices:
            self.add_vertice(key1, colour= col1)
        if key2 not in self.vertices:
            self.add_vertice(key2, colour= col2)
        self.vertices[key1].add_neighbour(self.vertices[key2], weight)
        
    def get_vertice(self, key):
        if key in self.vertices:
            return self.vertices[key]
        else:
            return None
        
    def get_vertices(self):
        return self.vertices.keys()
    

In [10]:
ward = Network(9)
for i in range(2,9):
    ward.generate_greedy(ward.start, 20, i, 'non-trans')

# ward.creating_all_data(20)

# print(ward.path_greedy(ward.start))

# print(ward.get_range(4))


# print(ward.path_random(ward.start))
# print(ward.path_dijkstra(ward.start,ward.end, True))


# # neighbour check
for vert in sorted(ward.vertices):
    print(vert , ward.vertices[vert].colour, [den.get_key() for den in ward.vertices[vert].get_neighbours()])

1 orange [2, 3]
2 green [4, 5]
3 red [4, 5]
4 white [6, 7]
5 black [6, 7]
6 orange [8, 9]
7 white [8, 9]
8 green [10, 11]
9 red [10, 11]
10 white [12, 13]
11 yellow [12, 13]
12 black [14, 15]
13 blue [14, 15]
14 green [16, 17]
15 purple [16, 17]
16 red [18, 19]
17 yellow [18, 19]
18 yellow [20, 21]
19 green [20, 21]
20 yellow []
21 yellow []
